In [17]:
# The Jupyter notebook magic command to automatically reload the source code when it is updated
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from playfab_manager.playfab_manager import PlayFabManager

Initialize playfab manager

In [22]:
playfab = PlayFabManager()

Entity token is valid


Get all players from the PlayFab

In [20]:
playfab.get_all_players()

If there are no error, play

datetime.datetime(2022, 11, 27, 19, 40, 52, tzinfo=datetime.timezone.utc)